In [1]:
import sqlalchemy as db
import pandas as pd
import plotly.express as px
import numpy as np

def read_credentials():
    with open('credentials.txt') as f:
        connection_string = f.read().strip()
    return connection_string

def connect_to(db_name):
    conn = None
    try:
        if db_name == 'postgres':
            connection_string = read_credentials()
            engine = db.create_engine(connection_string)
            conn = engine.connect()
            print("Successfully connected to the database")
        else:
            print(f"Database {db_name} is not supported.")
    except Exception as e:
        print(f"An error occurred while connecting to the database: {e}")
    
    return conn

# Задание 1

### Задание 1.1

In [6]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select 
    case
        when billing_country in ('Canada', 'USA') then 'North America'
        when billing_country in ('Argentina', 'Brazil', 'Chile') then 'South America'
        when billing_country in ('Austria', 'Belgium', 'Czech Republic', 'Denmark', 'Finland', 'France', 'Germany', 'Hungary', 'Ireland', 'Italy', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Spain', 'Sweden', 'United Kingdom') then 'Europe'
        when billing_country in ('India', 'Australia') then 'Asia'
    end as continent
    , billing_country as country
    , sum(total) as total_sales
    , count(*) as sales_number
from invoice
group by continent, billing_country
order by continent, billing_country;
"""

df = pd.read_sql_query(query, conn)

conn.close()

In [7]:
df

,continent,country,total_sales,sales_number
0,Asia,Australia,37.62,7
1,Asia,India,75.26,13
2,Europe,Austria,42.62,7
3,Europe,Belgium,37.62,7
4,Europe,Czech Republic,90.24,14
5,Europe,Denmark,37.62,7
6,Europe,Finland,41.62,7
7,Europe,France,195.10,35
8,Europe,Germany,156.48,28
9,Europe,Hungary,45.62,7


### Задание 1.2

In [25]:
fig = px.scatter(
    df,
    x= 'total_sales',
    y= 'sales_number',
    title= 'Соотношение суммы продаж на кол-во продаж',
    labels= {'total_sales': 'Сумма продаж', 'sales_number': 'Кол-во продаж'}
)

fig.show()

### Задание 1.3

In [28]:
fig = px.scatter(
    df,
    x= 'total_sales',
    y= 'sales_number',
    color= 'continent',
    title= 'Соотношение суммы продаж на кол-во продаж',
    labels= {'total_sales': 'Сумма продаж', 'sales_number': 'Кол-во продаж'},
    size= 'total_sales',
    color_discrete_map={
        'Asia':'yellow',
        'Europe': 'blue',
        'North America': 'red',
        'South America': 'green',
    }
)

fig.show()

### Задание 1.4

In [12]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select 
    case
        when i.billing_country in ('Canada', 'USA') then 'North America'
        when i.billing_country in ('Argentina', 'Brazil', 'Chile') then 'South America'
        when i.billing_country in ('Austria', 'Belgium', 'Czech Republic', 'Denmark', 'Finland', 'France', 'Germany', 'Hungary', 'Ireland', 'Italy', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Spain', 'Sweden', 'United Kingdom') then 'Europe'
        when i.billing_country in ('India', 'Australia') then 'Asia'
    end as continent
    , i.billing_country as country
    , sum(il.unit_price * il.quantity) as total_sales
    , count(il.*) as track_count
    , round(avg(il.unit_price * il.quantity), 2) as avg_sales_per_track
from invoice i
join invoice_line il on i.invoice_id = il.invoice_id
group by continent, i.billing_country
order by continent, i.billing_country;
"""

df2 = pd.read_sql_query(query, conn)

conn.close()

In [26]:
fig = px.scatter(
    df2,
    x="track_count",
    y="total_sales",
    text="country",
    color="continent",
    title="Соотношение суммы продаж и количества треков по странам",
    labels={"track_count": "Количество треков", "total_sales": "Сумма продаж ($)"},
    size="total_sales",
    hover_data={
        "country": True,
        "total_sales": ":$.2f",
        "track_count": ":d",
        "avg_sales_per_track": ":$.2f"
    },
    color_discrete_map={
        "Asia": "yellow",
        "Europe": "blue",
        "North America": "red",
        "Other": "gray"
    }
)

fig.show()

### Задание 1.5

In [27]:
fig = px.scatter(
    df2,
    x="track_count",
    y="total_sales",
    text="country",
    color="continent",
    title="Соотношение суммы продаж и количества треков по странам",
    labels={"track_count": "Количество треков", "total_sales": "Сумма продаж ($)"},
    size="total_sales",
    hover_data={
        "country": True,
        "total_sales": ":$.2f",
        "track_count": ":d",
        "avg_sales_per_track": ":$.2f"
    },
    color_discrete_map={
        "Asia": "yellow",
        "Europe": "blue",
        "North America": "red",
        "Other": "gray"
    }
)

fig.update_layout(
    title={
        'text': 'Соотношение суммы продаж и количества треков по странам',
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(
        size=30,
        color='gray'
    )
)

fig.show()

### Задание 1.6

In [29]:
fig = px.scatter(
    df2,
    x="track_count",
    y="total_sales",
    text="country",
    color="continent",
    title="Соотношение суммы продаж и количества треков по странам",
    labels={"track_count": "Количество треков", "total_sales": "Сумма продаж ($)"},
    size="total_sales",
    hover_data={
        "country": True,
        "total_sales": ":$.2f",
        "track_count": ":d",
        "avg_sales_per_track": ":$.2f"
    },
    color_discrete_map={
        "Asia": "yellow",
        "Europe": "blue",
        "North America": "red",
        "Other": "gray"
    }
)

fig.update_layout(
    title={
        "text": "Соотношение суммы продаж и количества треков по странам",
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    },
    title_font=dict(
        size=30,
        color="gray"
    ),
    xaxis=dict(
        tickfont=dict(
            size=14,
            color="gray"
        )
    ),
    yaxis=dict(
        tickfont=dict(
            size=14,
            color="gray"
        )
    )
)

fig.show()

# Задание 2

### Задание 2.1

In [3]:
connection_string = read_credentials()
engine = db.create_engine(connection_string)
conn = engine.connect()

query = """
select 
    g.name as genre_name
    , count(t.*) as tracks_number
from track t
join genre g on t.genre_id = g.genre_id
group by g.name
order by g.name;
"""

df3 = pd.read_sql_query(query, conn)

conn.close()

### Задание 2.2

In [8]:
fig = px.bar(
    df3,
    x='genre_name',
    y='tracks_number',
    title='Количество треков по жанрам',
    labels={'genre_name': 'Название жанра', 'tracks_number': 'Кол-во треков'}
)

fig.show()

### Задание 2.3

In [18]:
fig = px.bar(
    df3,
    x='genre_name',
    y='tracks_number',
    title='Количество треков по жанрам',
    labels={'genre_name': 'Название жанра', 'tracks_number': 'Кол-во треков'},
    color='genre_name',
    color_discrete_map={
        'Jazz':'green',
        'Rock': 'red',
        'Alternative & Punk': 'blue',
        'Alternative': 'gray',
        'Blues': 'gray',
        'Bossa Nova': 'gray',
        'Classical': 'gray',
        'Comedy': 'gray',
        'Drama': 'gray',
        'Easy Listening': 'gray',
        'Electronica/Dance': 'gray',
        'Heavy Metal': 'gray',
        'Hip Hop/Rap': 'gray',
        'Latin': 'gray',
        'Metal': 'gray',
        'Opera': 'gray',
        'Pop': 'gray',
        'R&B/Soul': 'gray',
        'Reggae': 'gray',
        'Rock And Roll': 'gray',
        'Sci Fi & Fantasy': 'gray',
        'Science Fiction': 'gray',
        'Soundtrack': 'gray',
        'TV Shows': 'gray',
        'World': 'gray',
        
    }
)

fig.show()

### Задание 2.4

In [19]:
fig = px.bar(
    df3,
    x='genre_name',
    y='tracks_number',
    title='Количество треков по жанрам',
    labels={'genre_name': 'Название жанра', 'tracks_number': 'Кол-во треков'},
    color='genre_name',
    color_discrete_map={
        'Jazz':'green',
        'Rock': 'red',
        'Alternative & Punk': 'blue',
        'Alternative': 'gray',
        'Blues': 'gray',
        'Bossa Nova': 'gray',
        'Classical': 'gray',
        'Comedy': 'gray',
        'Drama': 'gray',
        'Easy Listening': 'gray',
        'Electronica/Dance': 'gray',
        'Heavy Metal': 'gray',
        'Hip Hop/Rap': 'gray',
        'Latin': 'gray',
        'Metal': 'gray',
        'Opera': 'gray',
        'Pop': 'gray',
        'R&B/Soul': 'gray',
        'Reggae': 'gray',
        'Rock And Roll': 'gray',
        'Sci Fi & Fantasy': 'gray',
        'Science Fiction': 'gray',
        'Soundtrack': 'gray',
        'TV Shows': 'gray',
        'World': 'gray',
        
    }
)

fig.update_layout(
    showlegend=False
)

fig.show()

### Задание 2.5

In [20]:
fig = px.bar(
    df3,
    x='genre_name',
    y='tracks_number',
    title='Количество треков по жанрам',
    labels={'genre_name': 'Название жанра', 'tracks_number': 'Кол-во треков'},
    color='genre_name',
    color_discrete_map={
        'Jazz':'green',
        'Rock': 'red',
        'Alternative & Punk': 'blue',
        'Alternative': 'gray',
        'Blues': 'gray',
        'Bossa Nova': 'gray',
        'Classical': 'gray',
        'Comedy': 'gray',
        'Drama': 'gray',
        'Easy Listening': 'gray',
        'Electronica/Dance': 'gray',
        'Heavy Metal': 'gray',
        'Hip Hop/Rap': 'gray',
        'Latin': 'gray',
        'Metal': 'gray',
        'Opera': 'gray',
        'Pop': 'gray',
        'R&B/Soul': 'gray',
        'Reggae': 'gray',
        'Rock And Roll': 'gray',
        'Sci Fi & Fantasy': 'gray',
        'Science Fiction': 'gray',
        'Soundtrack': 'gray',
        'TV Shows': 'gray',
        'World': 'gray',
        
    }
)

fig.update_layout(
    showlegend=False,
    title={
        "text": "<i>Количество треков по жанрам</i>",
        "x": 0.5,
        "y": 0.9
    }
)

fig.show()

### Задание 2.6

In [21]:
fig = px.bar(
    df3,
    x='genre_name',
    y='tracks_number',
    title='Количество треков по жанрам',
    labels={'genre_name': 'Название жанра', 'tracks_number': 'Кол-во треков'},
    color='genre_name',
    color_discrete_map={
        'Jazz':'green',
        'Rock': 'red',
        'Alternative & Punk': 'blue',
        'Alternative': 'gray',
        'Blues': 'gray',
        'Bossa Nova': 'gray',
        'Classical': 'gray',
        'Comedy': 'gray',
        'Drama': 'gray',
        'Easy Listening': 'gray',
        'Electronica/Dance': 'gray',
        'Heavy Metal': 'gray',
        'Hip Hop/Rap': 'gray',
        'Latin': 'gray',
        'Metal': 'gray',
        'Opera': 'gray',
        'Pop': 'gray',
        'R&B/Soul': 'gray',
        'Reggae': 'gray',
        'Rock And Roll': 'gray',
        'Sci Fi & Fantasy': 'gray',
        'Science Fiction': 'gray',
        'Soundtrack': 'gray',
        'TV Shows': 'gray',
        'World': 'gray',
        
    }
)

fig.update_layout(
    showlegend=False,
    title={
        "text": "<i>Количество треков по жанрам</i>",
        "x": 0.5,
        "y": 0.9
    },
    plot_bgcolor="white",
    paper_bgcolor="white"
)

fig.show()